In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import INCLUDED_COUNTRIES
from autumn.projects.sm_covid2.common_school.runner_tools import multi_country_optimise, print_continuous_status, download_analysis

In [ ]:
iso3_list = list(INCLUDED_COUNTRIES['google_mobility'].keys())
len(iso3_list)

In [ ]:
N_CPUS = 32
N_OPTI_WORKERS = 8
assert N_CPUS % N_OPTI_WORKERS == 0

In [ ]:
def remote_multi_opti_task(bridge: springboard.task.TaskBridge, iso3_list: list):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running optimisation for {len(iso3_list)} countries.")

    n_parallel_opti_jobs = 2. * N_CPUS / N_OPTI_WORKERS

    multi_country_optimise(
        iso3_list=iso3_list, 
        num_workers = 8, 
        search_iterations = 15000, 
        parallel_opti_jobs = int(n_parallel_opti_jobs), 
        out_path=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(N_CPUS, 4, "compute")
task_kwargs = {
    "iso3_list": iso3_list,
}
tspec = springboard.TaskSpec(remote_multi_opti_task, task_kwargs)

In [ ]:
analysis_title = "multi_opti_15k"
run_path = springboard.launch.get_autumn_project_run_path("school_project", "multicountry", analysis_title)
run_path

In [ ]:
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

In [ ]:
runner.s3.get_status()

In [ ]:
# runner.sshr.run("sudo shutdown now")

In [ ]:
# wait function with status printing
# print_continuous_status(runner)

In [ ]:
# run_path = 'projects/school_project/multicountry/2023-08-30T1614-multi_opti_15k'

In [ ]:
download_analysis(run_path)